# Context Managers

In [1]:
# Old way
fp = open('/etc/hosts')
try:
    print(fp.read())
finally:
    fp.close()

# This file was automatically generated by WSL. To stop automatic generation of this file, add the following entry to /etc/wsl.conf:
# [network]
# generateHosts = false
127.0.0.1	localhost
127.0.1.1	theodin.localdomain	theodin
﻿
192.168.1.90	host.docker.internal
192.168.1.90	gateway.docker.internal
127.0.0.1	kubernetes.docker.internal

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters



In [2]:
with open('/etc/hosts') as fp:
    print(fp.read())

# This file was automatically generated by WSL. To stop automatic generation of this file, add the following entry to /etc/wsl.conf:
# [network]
# generateHosts = false
127.0.0.1	localhost
127.0.1.1	theodin.localdomain	theodin
﻿
192.168.1.90	host.docker.internal
192.168.1.90	gateway.docker.internal
127.0.0.1	kubernetes.docker.internal

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters



In [3]:
fp.closed

True

In [4]:
with open('/etc/hosts') as fp:
    raise KeyError
    print(fp.read())

KeyError: 

In [5]:
fp.closed

True

In [6]:
with open('/etc/hosts') as fp_i, open('/tmp/hosts', 'w') as fp_o:
    fp_o.write(fp_i.read())

In [7]:
with open('/etc/hosts') as fp_i:
    with open('/tmp/hosts', 'w') as fp_o:
        fp_o.write(fp_i.read())

In [8]:
fp_i.closed, fp_o.closed

(True, True)

In [9]:
with open('/tmp/hosts') as fp:
    print(fp.read())

# This file was automatically generated by WSL. To stop automatic generation of this file, add the following entry to /etc/wsl.conf:
# [network]
# generateHosts = false
127.0.0.1	localhost
127.0.1.1	theodin.localdomain	theodin
﻿
192.168.1.90	host.docker.internal
192.168.1.90	gateway.docker.internal
127.0.0.1	kubernetes.docker.internal

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters



In [10]:
fp = open('/tmp/hosts')
with fp as other_fp:
    print(other_fp.read())

# This file was automatically generated by WSL. To stop automatic generation of this file, add the following entry to /etc/wsl.conf:
# [network]
# generateHosts = false
127.0.0.1	localhost
127.0.1.1	theodin.localdomain	theodin
﻿
192.168.1.90	host.docker.internal
192.168.1.90	gateway.docker.internal
127.0.0.1	kubernetes.docker.internal

# The following lines are desirable for IPv6 capable hosts
::1     ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters



## Context manager protocol

In [12]:
class CM:
    
    def __enter__(self):
        print('Entering CM')
        return 'cabbage'  # the 'as' value ("with open(f) as fp..."")
        # most of the time we'd actually return self here
    
    def __exit__(self, ex_type, ex_val, ex_tb):
        # same order as sys.exc_info()
        print('Exiting CM')
        if ex_type == KeyError: 
            # Re-raise same exception
            print('Re-raise the key error')
            return False
        # Don't re-raise
        if ex_type is not None:
            print('Swallowing %s inside CM' % ex_type)
            return True

In [13]:
cm = CM()

In [14]:
cm

In [15]:
with cm as asvalue:
    print('Inside with statement', asvalue)

Entering CM
Inside with statement cabbage
Exiting CM


In [16]:
with CM():
    print('About to raise KeyError')
    raise KeyError
    print('This does not execute')

Entering CM
About to raise KeyError
Exiting CM
Re-raise the key error


KeyError: 

In [17]:
with CM():
    print('About to raise ValueError')
    raise ValueError
    print('This does not execute')

Entering CM
About to raise ValueError
Exiting CM
Swallowing <class 'ValueError'> inside CM


## Contextlib

In [18]:
import contextlib

In [ ]:
contextlib??

This is an abuse of generators, but it is useful...

In [19]:
@contextlib.contextmanager
def so_much_easier():
    print('Entering block')
    try:
        yield 'carrots'    # optional "as" value here
        print('Exiting block cleanly')
    except KeyError:
        print('got a KeyError')
        raise
    except:
        print('Exiting block with exception')

In [20]:
with so_much_easier() as as_value:    
    print('Inside block', as_value)

Entering block
Inside block carrots
Exiting block cleanly


In [21]:
with so_much_easier():
    print('Raising ValueError')
    raise ValueError
    print('Does not execute')

Entering block
Raising ValueError
Exiting block with exception


In [22]:
with so_much_easier():
    print('Raising KeyError')
    raise KeyError
    print('Does not execute')

Entering block
Raising KeyError
got a KeyError


KeyError: 

In [23]:
contextlib.suppress?

In [24]:
dct = {}
with contextlib.suppress(KeyError):
    print('the value is', dct['foo'])
    print('this line still does not run')

In [26]:
import sys

@contextlib.contextmanager
def reversit():
    old_write = sys.stdout.write
    sys.stdout.write = lambda text: old_write(text[::-1])
    try:
        yield
    finally:
        sys.stdout.write = old_write

In [27]:
with reversit():
    print('This is a reversed string')

gnirts desrever a si sihT


In [28]:
print('hello')

hello


In [29]:
with reversit():
    print('Lewis Carroll')
    print('JABBERWOCKY')

llorraC siweL
YKCOWREBBAJ


In [30]:
print('hello')

hello


# Use cases

- Files
- Multithreading/processing Lock

In [31]:
from threading import Lock
mylock = Lock()
with mylock:
    print('Serialize this code')

Serialize this code


Unittest.mock.patch

In [32]:
import unittest.mock
with unittest.mock.patch('sys.stdout') as mock_stdout:
    print('hi')

In [33]:
mock_stdout.write.assert_called()

In [34]:
mock_stdout.write.call_args_list

[call('hi'), call('\n')]

# Lab

Open [Context Managers Lab][context-lab]

[context-lab]: ./context-lab.ipynb